In [1]:
import glob
import pandas as pd

# Tratamento de dados coletados sobre proposições legislativas

Os dados referentes às proposições legislativas foram coletados manualmente a partir de arquivos estáticos disponíveis em https://dadosabertos.camara.leg.br/swagger/api.html#staticfile. Então, antes de iniciar seu tratamento, reuniremos todos os arquivos em um único dataframe pandas:

In [42]:
lista_proposicoes = glob.glob('../dados/proposicoes/*')

In [43]:
tipos_dados = {
    'id': object,
    'uri': object,
    'siglaTipo': object,
    'numero': object,
    'ano': int,
    'codTipo': object,
    'descricaoTipo': object,
    'ementa': object,
    'ementaDetalhada': object,
    'keywords': object,   
    'uriOrgaoNumerador': object,
    'uriPropAnterior': object,
    'uriPropPrincipal': object,
    'uriPropPosterior': object,
    'urlInteiroTeor': object, 
    'urnFinal': object, 
    'ultimoStatus_sequencia': object,
    'ultimoStatus_uriRelator': object, 
    'ultimoStatus_idOrgao': object,
    'ultimoStatus_siglaOrgao': object, 
    'ultimoStatus_uriOrgao': object,
    'ultimoStatus_regime': object, 
    'ultimoStatus_descricaoTramitacao': object,
    'ultimoStatus_idTipoTramitacao': object, 
    'ultimoStatus_descricaoSituacao': object,
    'ultimoStatus_idSituacao': object, 
    'ultimoStatus_despacho': object, 
    'ultimoStatus_url': object
}

tipo_data = ['dataApresentacao', 'ultimoStatus_dataHora']

In [44]:
lista_df = []

for proposicao in lista_proposicoes:
    df_proposicao = pd.read_csv(proposicao, sep=';', dtype=tipos_dados, parse_dates=tipo_data)
    lista_df.append(df_proposicao)

In [45]:
df_proposicao_1934_2021 = pd.concat(lista_df, axis=0, ignore_index=True)
df_proposicao_1934_2021.shape

(680358, 30)

## Seleção de dados referentes aos tipos de proposta legislativa desejados para análise

Selecionaremos apenas as propostas referentes aos seguintes tipos:
- Projeto de Decreto Legislativo [SF] (PDL)
- Projeto de Decreto Legislativo [CD] (PDC)
- Projeto de Decreto Legislativo [CN] (PDN)
- Projeto de Decreto Legislativo [SF] (PDS)
- Proposta de Emenda à Constituição (PEC)
- Projeto de Lei (PL)
- Projeto de Lei da Câmara (PLC)
- Projeto de Lei Complementar (PLP)
- Projeto de Lei de Conversão (PLV)
- Projeto de Resolução da Câmara dos Deputados (PRC)

In [46]:
tipos_proposicoes = ['PDS', 'PDC', 'PDN', 'PDS', 'PEC', 'PL', 'PLC', 'PLP', 'PLV', 'PRC']

In [47]:
df_proposicoes_tipos_desejados = df_proposicao_1934_2021[df_proposicao_1934_2021['siglaTipo'].isin(tipos_proposicoes)].copy()

In [48]:
df_proposicoes_tipos_desejados.shape

(152424, 30)

# Seleção de atributos desejados para análise

In [49]:
df_proposicoes = df_proposicoes_tipos_desejados[['id','siglaTipo','ano', 'codTipo', 'descricaoTipo',
       'ementa', 'ementaDetalhada', 'keywords']].copy()

In [50]:
df_proposicoes.shape

(152424, 8)

# Ajuste de valores faltantes

In [51]:
df_proposicoes.isnull().sum(axis = 0)

id                      0
siglaTipo               0
ano                     0
codTipo                 0
descricaoTipo           0
ementa                  5
ementaDetalhada    109844
keywords              924
dtype: int64

In [52]:
df_proposicoes[(df_proposicoes['ementa'].isnull()) & (df_proposicoes['ementaDetalhada'].isnull()) & (df_proposicoes['keywords'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [59]:
df_proposicoes[(df_proposicoes['ementa'].isnull())].head()

,id,siglaTipo,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords
59695,537493,PL,1946,139,Projeto de Lei,NaN,NaN,NaN
71533,347964,PL,1935,139,Projeto de Lei,NaN,NaN,NaN
424157,2123532,PLC,2017,465,Projeto de Lei da Câmara dos Deputados (SF),NaN,NaN,NaN
495917,2308959,PLV,2021,390,Projeto de Lei de Conversão,NaN,NaN,NaN
571961,605771,PL,1954,139,Projeto de Lei,NaN,NaN,NaN


In [60]:
df_proposicoes.dropna(axis=0, subset=['ementa'], inplace=True)

In [61]:
df_proposicoes.shape

(152419, 8)